##  Build a simple CNN model for classifying dog/cat  

Author: Anh Trung Tra   
Email: tratrunganh001@gmail.com

In [1]:
import tensorflow as tf
print(tf.__version__)

keras = tf.keras
%matplotlib inline

2.0.0-alpha0


In [2]:
# Create a dictionary describing the features.  
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_function(example_proto, size, is_augmented=True):
    """ TODO: Update description!
    """
    parsed_example = tf.io.parse_single_example(example_proto, image_feature_description)
    image_string = parsed_example['image_raw']
    label = parsed_example['label']
    
    # Don't use tf.image.decode_image, or the output shape will be undefined
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image_decoded, tf.float32) # convert to float values in [0, 1]
    
    if is_augmented: # augmented here!
        h = parsed_example['height']
        w = parsed_example['width']
        image = tf.image.random_flip_left_right(image)        
        image = tf.image.random_crop(image, [h, w, 3])
        image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        image = tf.clip_by_value(image, 0.0, 1.0)                
      
   # resize image to fit the input size
    resized_image = tf.image.resize_with_pad(image, size, size)
    
    # scale to [-1,1]
    resized_image = (resized_image - 0.5)*2.0

    return resized_image, label

train_dataset = (tf.data.TFRecordDataset('data/PetImages/train_19855.tfrecord')
                     .map(lambda ep: _parse_function(ep, 224, True), num_parallel_calls=4)
                     .shuffle(1000)
                     .batch(32)
                     .prefetch(1)  # make sure you always have one batch ready to serve
                )

val_dataset = (tf.data.TFRecordDataset('data/PetImages/val_4969.tfrecord')
                     .map(lambda ep: _parse_function(ep, 224, False), num_parallel_calls=4)
                     .batch(32)
                     .prefetch(1)  # make sure you always have one batch ready to serve
                )

In [3]:
mobilenet_conv = keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet', 
                                          include_top=False, 
                                          input_shape=(224, 224, 3))
# Freeze the layers except the last 4 layers
for layer in mobilenet_conv.layers:
    layer.trainable = False
mobilenet_conv.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [4]:
# Create the model
model = keras.models.Sequential()
# Add the vgg convolutional base model
model.add(mobilenet_conv)
# Add new layers
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(512, 1))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Conv2D(256, 1))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Conv2D(128, 1))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(2, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
batch_normalization_v2 (Batc (None, 7, 7, 1280)        5120      
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 7, 512)         655872    
_________________________________________________________________
activation (Activation)      (None, 7, 7, 512)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 256)         131328    
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         3

In [5]:
model.compile(optimizer=tf.optimizers.Adam(5e-6),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit_generator(train_dataset, 
                    epochs=100,
                    validation_data=val_dataset)

Epoch 1/100
621/621 [==============================] - 73s 117ms/step - loss: 0.4753 - accuracy: 0.8348 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
621/621 [==============================] - 70s 113ms/step - loss: 0.1933 - accuracy: 0.9436 - val_loss: 0.0999 - val_accuracy: 0.9712
Epoch 3/100
621/621 [==============================] - 71s 114ms/step - loss: 0.1387 - accuracy: 0.9519 - val_loss: 0.0837 - val_accuracy: 0.9738
Epoch 4/100
621/621 [==============================] - 68s 109ms/step - loss: 0.1180 - accuracy: 0.9574 - val_loss: 0.0798 - val_accuracy: 0.9710
Epoch 5/100
621/621 [==============================] - 65s 105ms/step - loss: 0.1128 - accuracy: 0.9568 - val_loss: 0.0788 - val_accuracy: 0.9716
Epoch 6/100
621/621 [==============================] - 66s 106ms/step - loss: 0.1088 - accuracy: 0.9581 - val_loss: 0.0782 - val_accuracy: 0.9726
Epoch 7/100
621/621 [==============================] - 66s 106ms/step - loss: 0.1015 - accuracy: 0.9621 - val_loss: 

Epoch 57/100
621/621 [==============================] - 64s 103ms/step - loss: 0.0520 - accuracy: 0.9817 - val_loss: 0.0614 - val_accuracy: 0.9761
Epoch 58/100
621/621 [==============================] - 64s 103ms/step - loss: 0.0522 - accuracy: 0.9803 - val_loss: 0.0612 - val_accuracy: 0.9763
Epoch 59/100
621/621 [==============================] - 64s 104ms/step - loss: 0.0527 - accuracy: 0.9802 - val_loss: 0.0610 - val_accuracy: 0.9769
Epoch 60/100
621/621 [==============================] - 64s 104ms/step - loss: 0.0501 - accuracy: 0.9811 - val_loss: 0.0610 - val_accuracy: 0.9767
Epoch 61/100
621/621 [==============================] - 64s 103ms/step - loss: 0.0506 - accuracy: 0.9812 - val_loss: 0.0608 - val_accuracy: 0.9779
Epoch 62/100
621/621 [==============================] - 64s 103ms/step - loss: 0.0496 - accuracy: 0.9819 - val_loss: 0.0609 - val_accuracy: 0.9775
Epoch 63/100
621/621 [==============================] - 66s 106ms/step - loss: 0.0500 - accuracy: 0.9818 - val_loss: 0

In [6]:
!pip install -q h5py pyyaml 

In [ ]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))